### CFD Exercise: Using C and ctypes

This example demostrates the use of C via the `ctypes` standard library module. An implementation of the Jacobi algorithm in C is provided in the file `jacobi.c`. The relevant function has the prototype
```c
int jacobi(int m, int n, int niter, double * psi);
```

If you want to try to write your own C code, you can delete or move the existing version. We suggest you use the same prototype.

In [ ]:
# Execute if you want to write your own `jacobi.c`
! mv jacaobi.c jacobi_ref.c

### Compile the shared library

Using `gcc` as the comipiler, we may compile the `jacobi.c` to a shared library via
```bash
$ gcc -c -fPIC jacobi.c
$ gcc -shared -o jacobi.so jacobi.o
```
The option `-fPIC` instructs the compiler to produce "position independent code"; this means that the object produced does not contain any absolute memory addresses. The associated shared library can then be loaded at runtime without the risk that it accesses memory asscoaited with any external code.

The cell below will do this via the shell escape. The result should be the shared object `jacobi.so`:

In [ ]:
! gcc -c -fPIC jacobi.c
! gcc -shared -o jacobi.so jacobi.o
! ls -l jacobi.so

### Load and describe the shared library in python

We now need to load the shared library, and describe the interface in the main python driver code `cfd.py`.

The first stage is to load the library
```python
import ctypes

lib = ctypes.cdll.LoadLibrary("jacobi.so")
```
Here we assume the shared object is in the same directory as the `cfd.py`; a real application may need to search the appropriate path to locate the library. A real application would also include appropriate erro checking.

Next, we need to describe the C prototype to python. This may be done via:
```python
# Return type
lib.jacobi.restype = ctypes.c_int

# Argument list
lib.jacobi.argtypes = [ctypes.c_int,
                       ctypes.c_int,
                       ctypes.c_int,
                       ctypes.POINTER(ctypes.c_double)]
```
The `ctypes.POINTER()` returns a `ctypes` pointer description corresponding to the C prototype. However, the `cfd.py` code operates on a `numpy.ndarray` object. A further stage is therefore required to achieve the correct result. This is
performed in the wrapper function:
```python
def jacobi(niter, psi):
    
    m, n = psi.shape
    data = psi.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    lib.jacobi(m, n, niter, data)
    
    return
```
This uses utilities provided by `ndarray` specifically to allow idenitication of appropriate `ctypes` descriptions.

It should now be possible to run and test the code.

In [ ]:
! ./cfd.py 1 1000
! diff velocity.dat ../verify/cfd_velocity_1_1000.dat